# RAG - Exemplo de implementação

**Retrieval-Augmented Generation (RAG)** é uma técnica que combina modelos de linguagem com mecanismos de recuperação de informações para melhorar a geração de texto.

O **RAG** revolucionou a interação, compreensão e geração de linguagem humana pelos sistemas de IA. Tornou os modelos de linguagem mais versáteis e inteligentes, sendo crucial para chatbots sofisticados e ferramentas complexas de criação de conteúdo. Uma das aplicações mais poderosas dos LLMs é a criação de chatbots sofisticados de perguntas e respostas (Q&A). Esses chatbots podem responder a perguntas sobre informações específicas usando RAG.

O LangChain tem vários componentes projetados para ajudar a criar aplicativos de perguntas e respostas e aplicativos RAG de forma mais geral.

## Instalação das bibliotecas

Executar este comando, caso as bibliotecas não foram instaladas no ambiente.

In [1]:
!pip install -q transformers einops accelerate bitsandbytes
!pip install -q langchain langchain_community langchain-huggingface langchainhub langchain_chroma

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Ge

## Importações necessárias

In [2]:
import torch
import os
import getpass

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline

from langchain.prompts import PromptTemplate
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

## Adicionando o Token do huggingface

In [3]:
os.environ["HF_TOKEN"] = getpass.getpass()

··········


## Carregando a LLM

In [4]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

pipe = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.1,
    max_new_tokens=500,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
)
llm = HuggingFacePipeline(pipeline=pipe)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Device set to use cuda:0


## Template LLAMA 3 sem RAG

In [6]:
template = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Você é um assistente virtual prestativo e está respondendo perguntas gerais.
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
{pergunta}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

template

'\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente virtual prestativo e está respondendo perguntas gerais.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n{pergunta}\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n'

## Prompt sem RAG

In [7]:
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['pergunta'], input_types={}, partial_variables={}, template='\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente virtual prestativo e está respondendo perguntas gerais.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n{pergunta}\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n')

## Chain utilizando prompt sem RAG

In [9]:
chain = prompt | llm

## Invoke sem RAG

Como primeiro exemplo, vamos usar o dia de hoje. Aqui é algo onde o contexto através de RAG pode ser útil já que está totalmente fora do alcance da LLM retornar o dia hoje, é algo que ela simplesmente não tem como saber sozinha

In [10]:
chain.invoke({"pergunta": "Que dia é hoje?"})

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'Peço desculpas, mas como sou um assistente virtual, não tenho acesso a informações em tempo real sobre a data atual. No entanto, posso ajudá-lo com outras coisas!'

## Template LLAMA 3 sem RAG
Para implementar o RAG, devemos reservar um espaço no template do prompt para que seja alocado nessa parte o contexto que queremos usar

Basearemos nesse template que está hospedado no Hub do LangSmith https://smith.langchain.com/hub/rlm/rag-prompt É um prompt bastante usado para esse objetivo

Mais tarde ensinaremos como puxar diretamente os prompts de lá sem precisar copiar e colar. Mas agora, fazemos assim pois queremos adequar ao template do modelo Llama 3

In [11]:
template_rag = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Você é um assistente virtual prestativo e está respondendo perguntas gerais.
Use os seguintes pedaços de contexto recuperado para responder à pergunta.
Se você não sabe a resposta, apenas diga que não sabe. Mantenha a resposta concisa.
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Pergunta: {pergunta}
Contexto: {contexto}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

## Prompt com RAG

In [12]:
prompt_rag = PromptTemplate.from_template(template_rag)
print(prompt_rag)

input_variables=['contexto', 'pergunta'] input_types={} partial_variables={} template='\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente virtual prestativo e está respondendo perguntas gerais.\nUse os seguintes pedaços de contexto recuperado para responder à pergunta.\nSe você não sabe a resposta, apenas diga que não sabe. Mantenha a resposta concisa.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nPergunta: {pergunta}\nContexto: {contexto}\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n'


## Definindo o contexto

Agora vamos usar uma função do python que nos retorna o dia

In [13]:
from datetime import date

dia = date.today()
print(dia)

2025-02-06


In [14]:
contexto = "Você sabe que hoje é dia '{}'".format(dia)
print(contexto)

Você sabe que hoje é dia '2025-02-06'


## Criação da Chain / Geração

Dica: Se a pergunta for muito vaga e referente a algo que o modelo possa saber (por exemplo, algo muito conhecido) ele pode alucinar um pouco na resposta as vezes. Para evitar alucinar, pode colocar no template o seguinte: "Responda a pergunta com base apenas no contexto". Lembre-se que você pode sempre tentar aperfeiçoar o prompt para melhorar os resultados

In [15]:
chain_rag = prompt_rag | llm | StrOutputParser()

pergunta = "Que dia é hoje? Retorne a data em formato dd/mm/yyyy"

res = chain_rag.invoke({"pergunta": pergunta, "contexto": contexto})
res

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'Hoje é 06/02/2025.'

## RAG - Explorando dados de faturamento de uma empresa

Vamos pegar um exemplo mais concreto. Vamos supor que queremos usar o LLMs para responder duvidas de um documento ou uma planilha que contém informações sobre um empresa. Esses dados são privados e portanto os modelos não tem como saber (e mesmo que fossem públicos, bom lembrar que a precisão seria comprometida, sem falar que dados muitos recentes não estariam inclusos)

In [16]:
chain_rag = prompt_rag | llm | StrOutputParser()

contexto = """Faturamento trimestral:
1º: R$42476,40
2º: R$46212,97
3º: R$41324,56
4º: R$56430,24"""

#pergunta = "Qual é o faturamento do segundo trimestre?"
pergunta = "Qual trimestre teve o maior faturamento?"

chain_rag.invoke({
  "contexto": contexto,
  "pergunta": pergunta
})

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'O trimestre com o maior faturamento foi o 4º, com R$56430,24.'

## Depuração / Debugging

Assim como na construção de qualquer tipo de software, em algum momento você precisará depurar ao construir com LLMs. Isso porque uma chamada de modelo falhará ou a saída do modelo será malformatada, ou haverá algumas chamadas de modelo aninhadas e não ficará claro onde ao longo do caminho uma saída incorreta foi criada.

Há três métodos principais para depuração:

Modo Debug: Isso adiciona instruções de registro para TODOS os eventos em sua cadeia. Modo Verbose: Isso adiciona instruções de impressão para eventos "importantes" em sua cadeia. Rastreamento com LangSmith: Isso registra eventos no LangSmith para permitir a visualização lá.

Veremos o método com LangSmith mais tarde.

Agora, vamos aprender como fazer isso ativando o modo debug

https://python.langchain.com/v0.2/docs/how_to/debugging/

In [17]:
from langchain.globals import set_debug
set_debug(True)

Definir a depuração = True com que todos os componentes do LangChain com suporte a callback (chains, modelos, agentes, tools, retrievers, etc.) imprimam as entradas que recebem e as saídas que geram. Esta é a configuração mais detalhada e registrará totalmente as entradas e saídas brutas

In [18]:
pergunta = "Qual trimestre teve o menor faturamento?"

chain_rag.invoke({
  "contexto": contexto,
  "pergunta": pergunta
})

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "contexto": "Faturamento trimestral:\n1º: R$42476,40\n2º: R$46212,97\n3º: R$41324,56\n4º: R$56430,24",
  "pergunta": "Qual trimestre teve o menor faturamento?"
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "contexto": "Faturamento trimestral:\n1º: R$42476,40\n2º: R$46212,97\n3º: R$41324,56\n4º: R$56430,24",
  "pergunta": "Qual trimestre teve o menor faturamento?"
}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:HuggingFacePipeline] Entering LLM run with input:
{
  "prompts": [
    "<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente virtual prestativo e está respondendo perguntas gerais.\nUse os seguintes pedaços de contexto recuperado para responder à pergunta.\nSe você não sabe a resposta, apenas diga que não sabe. Mantenha a 

'O trimestre com o menor faturamento foi o 3º, com R$41324,56.'

In [19]:
set_debug(False)

Alternativamente, você pode usar um serviço externo como o LangSmith para rastrear melhor: https://smith.langchain.com/ LangSmith irá capturar os detalhes internos em cada etapa

## Aplicação para RAG com contextos maiores

Nosso próximo exemplo vai consistir de acessar uma página na internet e usar RAG para fazer a LLM "conversar" com ela, usando o seu conteúdo como contexto e assim responder nossas dúvidas. Ou seja, basicamente estaremos fazendo um Webscraping e utilizando o conteúdo lido como contexto

A informação que queremos saber seria impossível com esse modelo que estamos usando nesse exemplo (llama 3.0) pois os dados de treinamento mais recentes são de 2023, portanto, não teria como saber nada do que aconteceu em 2024. Isso significa que o modelo vai retornar algo como "não aconteceu ainda" ou "não tenho como saber esse tipo de informação". Em alguns casos, dependendo do prompt, e ele pode tentar responder algo plausível mas levemente alucinado - portanto ainda não seria totalmente coerente com nossa dúvida

Mesmo que fosse uma informação anterior aos dados de treinamento, ainda assim o RAG poderia ser usado, pelas razões que comentamos, principalmente por garantir resultados mais precisos

## Etapas de Indexação

### 1 - Carregar o conteúdo

Primeiramente, precisamos carregar o conteúdo desejado.
Como nesse exemplo queremos acessar uma página web (que contém as informações de nossa dúvida)

Para isso:

* Usamos o DocumentLoaders, que são objetos que carregam dados de uma fonte e retornam uma lista de `Documents`. Nesse contexto, um Document é um objeto com `page_content` (retornado em string) e `metadata` (retornado em dicionário).

O LangChain possui suporte a centenas de formas de carregadores de documentos, para os mais diversos meios e formatos ou extensões de arquivos (PDF, CSV, etc.)

> Confira aqui: https://python.langchain.com/v0.2/docs/integrations/document_loaders/

* Como queremos carregar os dados de uma página web nós importamos o WebBaseLoader, que usa urllib para carregar HTML de URLs e a biblioteca BeautifulSoup para converter em texto.

In [20]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

* Você pode customizar a conversão HTML para texto passando parâmetros para o parser do BeautifulSoup via bs_kwargs.

  * Poderiamos especificar tags HTML com classes específicas, para puxar apenas o conteúdo que está dentro dela - nesse caso, poderia ser o bloco HTML que contém o conteúdo do artigo, que é onde estão as informações relevantes - e então todas as outras partes da página são removidas (que contém informações não relevantes, como cabeçalho do site, rodapé, etc.).
  
  * Para descobrir a classe: selecione a opção "Inspecionar elemento" que aparece ao clicar com o botão direito na página. ou "Ferramentas de desenvolvedor > Inspecionar" (Ctrl + Shift + I)

Neste caso, não usaremos pois é uma página que não tem tanto conteúdo fora do container principal do site, portanto não acaba sendo um problema.

Mas você pode filtrar com base nas classes para que pegue somente o texto dentro do bloco principal de conteúdo da página

In [21]:
loader = WebBaseLoader(web_paths = ("https://www.bbc.com/portuguese/articles/cd19vexw0y1o",),)
docs = loader.load()

In [22]:
len(docs[0].page_content)

11941

Acima usamos o len() para exibir comprimento (em caracteres) do conteúdo da primeira página carregada

Podemos dar print no docs, o que mostrará o conteúdo completo.

Usamos isso para verificar a quantidade de conteúdo carregado e visualizar uma amostra inicial do texto carregado. Isso é útil para garantir que o carregamento do conteúdo e a filtragem dos elementos HTML relevantes foram realizados corretamente

In [23]:
docs

[Document(metadata={'source': 'https://www.bbc.com/portuguese/articles/cd19vexw0y1o', 'title': 'Oscar 2024: confira todos os ganhadores dos prêmios da Academia de Hollywood  - BBC News Brasil', 'description': "'Oppenheimer' foi o grande vencedor da noite com sete estatuetas, incluindo o prêmio de melhor filme, melhor diretor e melhor ator.", 'language': 'pt-br'}, page_content="Oscar 2024: confira todos os ganhadores dos prêmios da Academia de Hollywood  - BBC News BrasilBBC News, BrasilVá para o conteúdoSeçõesNotíciasBrasilInternacionalEconomiaSaúdeCiênciaTecnologiaVídeosBBC LêNotíciasBrasilInternacionalEconomiaSaúdeCiênciaTecnologiaVídeosBBC LêOscar 2024: confira todos os ganhadores dos prêmios da Academia de HollywoodCrédito, Getty ImagesLegenda da foto, Robert Downey Jr., Da'Vine Hoy Randolph, Emma Stone e Cillian Murphy com suas respectivas estatuetas do OscarArticle informationAuthor, Leire VentasRole,  Correspondente da BBC News Mundo em Los AngelesTwitter, @leire_ventas11 março 

A linha de baixo imprime os primeiros 300 caracteres do conteúdo da primeira página carregada. A sintaxe [:300] é usada para obter uma substring dos primeiros 300 caracteres da string page_content. Pode ser uma maneira rápida de verificação

In [24]:
print(docs[0].page_content[:300])

Oscar 2024: confira todos os ganhadores dos prêmios da Academia de Hollywood  - BBC News BrasilBBC News, BrasilVá para o conteúdoSeçõesNotíciasBrasilInternacionalEconomiaSaúdeCiênciaTecnologiaVídeosBBC LêNotíciasBrasilInternacionalEconomiaSaúdeCiênciaTecnologiaVídeosBBC LêOscar 2024: confira todos o


### 2 - Divisão em pedaços de texto / Split

Nosso documento carregado tem mais de 10 mil caracteres, o que seria muito grande para passar como contexto usando o método que fizemos até então.

Por exemplo pro GPT-4, o tamanho da janela de contexto do é de cerca de 8.000 tokens, o que equivale aproximadamente a 32.000 caracteres. Então na verdade para esse exemplo até poderia caber a postagem completa em sua janela de contexto, porém as LLMs geralmente terão dificuldade em localizar informações dentro de entradas tão longas.

No entanto, verá que é bem fácil um documento passar esse limite de tokens, o que fará com que o modelo não consiga processar tudo, já que é uma sequência de texto muito longa para caber na janela de contexto da grande maioria dos modelos.

Portanto, é uma boa prática fazer a divisão em documentos tão longos, assim já temos o código pronto e preparado para conseguir processar documentos muito maiores.

> Processo de divisão

* Dividiremos o `Document` em pedaços para incorporação e armazenamento vetorial. Isso deve nos ajudar a recuperar apenas os bits mais relevantes da postagem do blog em tempo de execução.

* Usamos o RecursiveCharacterTextSplitter, que dividirá recursivamente o documento usando separadores comuns, como novas linhas, até que cada pedaço tenha o tamanho apropriado. Este é o divisor de texto recomendado para casos de uso de texto gerais ou genérico.

* Neste caso, dividiremos em pedaços (chunks) de 1000 caracteres. Para isso usaremos o parâmetro `chunk_size`. Um tamanho de chunk menor resultará em mais pedaços, enquanto um tamanho de chunk maior resultará em menos pedaços.

* Com a divisão, é necessário fazer o que chamamos de sobreposição (overlap). Essa sobreposição ajuda a evitar que uma declaração seja separada do seu contexto importante. Aqui nesse exemplo vamos definir 200 caracteres de sobreposição entre os pedaços - usamos o parâmetro `chunk_overlap`.  Uma sobreposição de chunks maiores resultará em mais pedaços compartilhando caracteres comuns, enquanto uma sobreposição de chunks menores resultará em menos pedaços compartilhando caracteres comuns.

 * Escolhendo o valor de `chunk_size` e o `chunk_overlap` - geralmente recomenda-se experimentar diferentes valores, pois depende do problema específico que você está tentando resolver. No entanto, em geral, é uma boa ideia usar um tamanho de chunk/bloco pequeno para tarefas que exigem uma visão detalhada do texto (fine-grained view) e um tamanho de bloco maior para tarefas que exigem uma visão mais "holística" do texto (ou seja, uma visão mais geral, do todo) .


* E definimos `add_start_index=True` para que o índice de caracteres no qual cada Documento dividido começa dentro do Documento inicial seja preservado como atributo de metadados "start_index".

> Para outros tipos de transformações possíveis de serem feitos com a biblioteca LangChain, consulte a documentação https://python.langchain.com/v0.2/docs/integrations/document_transformers/

In [25]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200, add_start_index = True)
splits = text_splitter.split_documents(docs)

In [26]:
len(splits)

15

In [27]:
splits[1]

Document(metadata={'source': 'https://www.bbc.com/portuguese/articles/cd19vexw0y1o', 'title': 'Oscar 2024: confira todos os ganhadores dos prêmios da Academia de Hollywood  - BBC News Brasil', 'description': "'Oppenheimer' foi o grande vencedor da noite com sete estatuetas, incluindo o prêmio de melhor filme, melhor diretor e melhor ator.", 'language': 'pt-br', 'start_index': 800}, page_content="melhor filme, melhor diretor para Christopher Nolan, melhor ator para Cillian Murphy e melhor ator coadjuvante para Robert Downey Jr.Em uma cerimônia com poucas surpresas, na qual a maior parte das apostas da crítica e do público se concretizaram, Pobres Criaturas também conquistou vários prêmios.Uma das quatro estatuetas que o filme do grego Yorgos Lanthimos arrebatou foi graças a Emma Stone, que ganhou o prêmio de melhor atriz por interpretar a protagonista Bella Baxter.O Oscar de Da'Vine Joy Randolph como melhor atriz coadjuvante foi o único de Os Rejeitados; e o de Billie Eislish, que receb

Podemos trabalhar com metadata usando RAG por exemplo para rastrear onde a LLM obteve a resposta (útil caso estejamos usando várias fontes/arquivos diferentes para RAG)

In [28]:
splits[1].metadata

{'source': 'https://www.bbc.com/portuguese/articles/cd19vexw0y1o',
 'title': 'Oscar 2024: confira todos os ganhadores dos prêmios da Academia de Hollywood  - BBC News Brasil',
 'description': "'Oppenheimer' foi o grande vencedor da noite com sete estatuetas, incluindo o prêmio de melhor filme, melhor diretor e melhor ator.",
 'language': 'pt-br',
 'start_index': 800}

Mais sobre os text splitters https://python.langchain.com/v0.2/docs/how_to/#text-splitters

### 3 - Armazenamento

Agora precisamos indexar nossos pedaços de texto para que possamos pesquisá-los. A maneira mais comum de fazer isso é incorporar o conteúdo de cada divisão de document e inserir esses embeddings em um banco de dados de vetores (ou armazenamento de vetores).

Quando queremos pesquisar em nossas divisões, pegamos uma consulta de pesquisa de texto, a incorporamos e realizamos algum tipo de pesquisa de "similaridade" para identificar as divisões armazenadas com os embeddings mais semelhantes ao nosso embedding de consulta.

A medida de similaridade mais simples é a similaridade de cosseno — medimos o cosseno do ângulo entre cada par de embeddings (que são vetores de alta dimensão).

#### Embeddings

Embedding é uma representação numérica de um texto. Olhando para eles, não são nada além de números, mas por trás dos panos, eles possuem uma relação entre si. Isso significa que palavras ou frases com significados semelhantes terão embeddings próximos uns dos outros em um espaço vetorial. Através dos embeddings conseguiremos procurar por itens similares (nesse caso, palavras)

> [ Mais explicações nos slides com título "Embeddings" ]

Podemos incorporar e armazenar todas as nossas divisões de documentos em um único comando usando o armazenamento de vetores Chroma

Mas antes, precisamos escolher qual embedding usar.

> Modelos de embedding open source
 * A principal vantagem de uso de modelo é que, assim como a LLM, modelo open source podemos rodar de graça e localmente offline

 * No Hugging Face podemos encontrar vários:
  * https://huggingface.co/models?sort=trending&search=embeddings
  * https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=trending

 * Vamos escolher o "sentence-transformers/all-mpnet-base-v2", que costuma ser bom e funcionou bem para nossos resultdos.

 * Mas você pode testar outros também se desejar.

> Modelos de embedding proprietários

 * Você també pode encontrar modelos de embedding em soluções proprietárias com o OpenAI, que disponibiliza também modelo de embedding como o text-embedding-3-large (mais info, consulte [aqui](https://platform.openai.com/docs/guides/embeddings)). O valor do modelo de embedding é bem mais barato, para consultar veja [openai.com/api/pricing/](https://openai.com/api/pricing/)

 * Caso opte por essa opção, pode pular essa parte da definição da variável contendo o modelo de embeddings, pule direto para o código que faz o armazenamento como Chroma, onde é definido o OpenAIEmbeddings().

> Além do HuggingFaceEmbeddings e do OpenAIEmbeddings, há várias outras classes disponíveis para você implementar outros tipos de modelos de embedding, consulte: https://python.langchain.com/v0.2/docs/integrations/text_embedding/



In [29]:
hf_embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [30]:
input_test = "Um teste apenas"
result = hf_embeddings.embed_query(input_test)

In [31]:
len(result)

768

In [32]:
print(result)

[-0.026895053684711456, -0.011064697988331318, -0.04531998932361603, -0.0013973648892715573, 0.042436011135578156, -0.014201739802956581, 0.023354941979050636, 0.06011558324098587, 0.061521951109170914, 0.006502094678580761, 0.008159150369465351, -0.030567463487386703, 0.002060323255136609, 0.01296155247837305, 0.004251229576766491, 0.0036631699185818434, -0.026755528524518013, 0.029737044125795364, -0.009770004078745842, -0.046504564583301544, -0.028108958154916763, 0.00016858700837474316, -0.024811016395688057, -0.011828277260065079, 0.08182830363512039, 0.0014993291115388274, 0.013264901004731655, -0.06242178753018379, -0.0012287781573832035, 0.02897716872394085, -0.029528986662626266, -0.025696074590086937, 0.003377106273546815, -0.028868991881608963, 1.518518047305406e-06, -0.03882475197315216, -0.019842026755213737, -0.01765800639986992, -0.008112344890832901, -0.02513054944574833, 0.025749174878001213, 0.11367511004209518, -0.006133120507001877, -0.017986472696065903, -0.0454115

#### Armazenando no banco de dados vetorial

Nós estamos escolhendo o **Chroma** como nosso banco de dados de vetores por ser bastante usado e versátil.

Assim como os outros componentes, o wrapper VectorStore possui diversas outras classes além do Chroma, assim você pode armazenar os vetores usando outro método ou serviço de sua preferência: https://python.langchain.com/v0.2/docs/integrations/vectorstores/

* Outras opções: FAISS, Pinecone, Qdrant, etc.

In [33]:
vectorstore = Chroma.from_documents(documents=splits, embedding=hf_embeddings)

# caso fossemos usar os embeddings da Open AI, basta mudar o método, passando direto conforme abaixo
# vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

Isso conclui a parte de Indexação de nessa pipeline de RAG.

Até este ponto, já temos um repositório de vetores consultável contendo os conteúdos em blocos.

Dada uma pergunta feita, devemos ser capazes de retornar os snippets de conteúdo que respondem à pergunta.

## Etapas de Recuperação e geração

### 4 - Configurando o recuperador de texto / Retriever

Nessa aplicação que estamos montando, queremos que receba uma pergunta do usuário, busque documentos relevantes para essa pergunta, passe os documentos recuperados e a pergunta inicial para um modelo e retorne uma resposta.

Primeiro, precisamos definir nossa lógica para buscar nos documentos. O LangChain define uma interface chamada Retriever, que envolve um índice capaz de retornar `Documents` relevantes dado uma consulta em forma de string.

O tipo mais comum de Retriever é o [VectorStoreRetriever](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStoreRetriever.html), que utiliza as capacidades de busca por similaridade de um vector store para facilitar a recuperação.

Qualquer VectorStore pode ser facilmente transformado em um Retriever com `VectorStore.as_retriever()`. Quanto aos parâmetros:

* search_type - é o tipo de pesquisa que será realizada. Lembrando que o que estamos buscando basicamente é similaridade, dentro do documento, como uma página web ou pdf por exemplo

* search_kwargs: "k" - Também podemos limitar o número de documentos k retornados pelo recuperador. Aqui você pode testar bastante os valores e verificar os resultados. 6 é um valor considerado meio padrão - e que funcionou bem para nossos testes - mas você pode variar ele para mais ou para menos, testar qual valor fica melhor pro seu caso
(aqui numero menor como 3 não deu conta de algumas perguntas, você pode diminuir e testar para comprovar)

In [34]:
retriever = vectorstore.as_retriever(search_type = "similarity", search_kwargs={"k": 6})

> Explorando mais os retrievers  

* Outros métodos de busca - Por padrão, o retriever de armazenamento de vetores usa pesquisa de similaridade. Se o armazenamento de vetores que estiver usando suportar pesquisa de "relevância marginal máxima" (baseado no MMR - Maximum marginal relevance retrieval), você pode especificar isso como o tipo de pesquisa, basta trocar "similarity" por "mmr"

 * O [MMR](https://www.cs.cmu.edu/~jgc/publication/The_Use_MMR_Diversity_Based_LTMIR_1998.pdf) (Relevância Marginal Máxima) seleciona por relevância e diversidade entre os documentos recuperados para evitar passar em contexto duplicado. Ou seja, é um método usado para evitar redundância ao recuperar itens relevantes para uma consulta. Em vez de simplesmente recuperar os itens mais relevantes (que muitas vezes podem ser muito semelhantes entre si), o MMR garante um equilíbrio entre relevância e diversidade nos itens recuperados.

* MultiQueryRetriever - permite gerar variações da pergunta de entrada para melhorar a taxa de acerto de recuperação ([mais info](https://python.langchain.com/v0.2/docs/how_to/MultiQueryRetriever/))

* MultiVectorRetriever - já esse gera variantes dos embeddings, também para melhorar a taxa de acerto de recuperação ([mais info](https://python.langchain.com/v0.2/docs/how_to/multi_vector/))

### 5 - Geração

Vamos juntar tudo em uma chain, que irá: pegar uma pergunta -> recupera documentos relevantes -> construir um prompt -> passar para o modelo -> processar a saída.


In [35]:
template_rag

'\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente virtual prestativo e está respondendo perguntas gerais.\nUse os seguintes pedaços de contexto recuperado para responder à pergunta.\nSe você não sabe a resposta, apenas diga que não sabe. Mantenha a resposta concisa.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nPergunta: {pergunta}\nContexto: {contexto}\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n'

In [36]:
prompt_rag = PromptTemplate(
    input_variables=["contexto", "pergunta"],
    template=template_rag,
)
prompt_rag

PromptTemplate(input_variables=['contexto', 'pergunta'], input_types={}, partial_variables={}, template='\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente virtual prestativo e está respondendo perguntas gerais.\nUse os seguintes pedaços de contexto recuperado para responder à pergunta.\nSe você não sabe a resposta, apenas diga que não sabe. Mantenha a resposta concisa.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nPergunta: {pergunta}\nContexto: {contexto}\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n')

> Formação de documentos

A função abaixo recebe uma lista de documentos (docs) e retorna uma única string onde o conteúdo de cada documento é concatenado com duas quebras de linha entre eles. No contexto de processamento de texto para uma pipeline de RAG, essa função formata os documentos recuperados de modo que seu conteúdo possa ser passado de maneira estruturada para um modelo de linguagem, facilitando a geração de respostas baseadas nos dados dos documentos

In [37]:
def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

> Definição da chain

Podemos deixar a chain desse modo, com quebra de linha, pois dará mais legibilidade (também graças à sintaxe do LCEL, conforme citado) já que agora temos mais elos / componentes em nossa chain

Aqui usamos o método RunnablePassthrough, que por si só permite que você passe entradas inalteradas

In [38]:
chain_rag = ({"contexto": retriever | format_docs, "pergunta": RunnablePassthrough()}
             | prompt_rag
             | llm
             | StrOutputParser())

### Teste sem RAG

In [39]:
chain.invoke("Qual filme ganhou mais oscars na premiação de 2024?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'Infelizmente, a premiação dos Oscars de 2024 ainda não ocorreu. A cerimônia anual dos Academy Awards é realizada todos os anos em fevereiro ou março, e o resultado do concurso é divulgado nessa época. Por isso, não há um filme que tenha ganhado mais Oscars na premiação de 2024, pois ela ainda não aconteceu.'

### Teste com RAG

In [40]:
chain_rag.invoke("Qual filme ganhou mais oscars na premiação de 2024?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'O filme "Oppenheimer" ganhou sete estatuetas, incluindo o prêmio de melhor filme, melhor diretor para Christopher Nolan, melhor ator para Cillian Murphy e melhor ator coadjuvante para Robert Downey Jr. na premiação do Oscar 2024.'

In [41]:
chain_rag.invoke("Quem ganhou o premio de melhor ator?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'A pergunta é: Quem ganhou o prêmio de melhor ator?\n\nA resposta é: Cillian Murphy, que ganhou o prêmio de melhor ator por seu papel em "Oppenheimer".'

> Limpando o vector store


O comando vectorstore.delete_collection() é usado no contexto de gerenciamento de um repositório de vetores, que em sua aplicação é uma estrutura semelhante a um banco de dados que armazena representações vetoriais de dados de texto

In [42]:
vectorstore.delete_collection()

* Este comando instrui o vectorstore a deletar toda a coleção de dados que ele contém. Uma coleção refere-se ao conjunto de todos os documentos (trechos de texto) e suas representações vetoriais correspondentes que foram indexados e armazenados no vectorstore.

* Observação sobre impacto na aplicação: Ao executar vectorstore.delete_collection(), você remove efetivamente todos os dados indexados do repositório de vetores. Esta ação limpa o banco de dados, o que pode ser necessário durante redefinições do sistema, atualizações ou quando você deseja limpar dados antigos para abrir espaço para novos dados a serem processados e armazenados.

Esta função é crítica para manter a limpeza e a relevância dos dados em aplicações que dependem de conjuntos de dados dinâmicos ou que requerem atualizações periódicas na sua estrutura de informações. Ela garante que o sistema possa ser atualizado ou reestruturado sem que dados residuais de operações anteriores interfiram nas novas operações.

## Resumo sobre RAG e LLMs usando LangChain

Resumo geral do pipeline que montamos acima:

1. Carregar o conteúdo da página ou PDF ou outro arquivo/mídia
2. Divisão em Chunks: O primeiro passo é dividir os documentos em pequenos pedaços, ou chunks.
3. Armazenamento e Transformação em Embeddings: Esses chunks são transformados em embeddings, que são representações vetoriais dos textos. Os embeddings são armazenados em um banco de dados vetorial (vector database).
4. Uso de Retriever: O banco de dados vetorial fornece um retriever (recuperador de informações) que busca os chunks mais relevantes com base em um algoritmo de similaridade.
5. Junção do contexto ao prompt e geração do resultado final

**Fluxo de Trabalho:**
* Primeira Chain (LLM + Prompt): O modelo de linguagem (LLM) recebe um prompt inicial e gera uma resposta.
* Integração com Retriever: A resposta inicial é combinada com o retriever para formar outro elo na cadeia.

**Funcionamento do Sistema:**
* Input como Embedding: O input fornecido é convertido em um embedding.
* Busca no banco de dados: O embedding do input é usado para buscar os chunks mais relevantes no banco de dados.
* Execução do LLM: Os chunks encontrados são utilizados pelo LLM para gerar a resposta final.
* Rastreamento de chunks utilizados: É possível acessar quais chunks foram usados na geração da resposta.

